In [102]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import pylab as pl
from matplotlib import collections  as mc
import time

In [103]:
from numba import cuda
from numba import *
from numba import jit

In [104]:
np.random.seed(1337)

In [105]:
# Load our data
cities = pd.read_csv("cities.csv")
sample = pd.read_csv("sample_submission.csv")

In [106]:
# Plots Tour
def plot_tour(maps, tour, endpoint=None):
    lines = [[(maps.X[tour[i]],maps.Y[tour[i]]),(maps.X[tour[i+1]],maps.Y[tour[i+1]])] for i in range(0,len(maps)-1)]
    lc = mc.LineCollection(lines, linewidths=2)
    fig, ax = pl.subplots(figsize=(20,20))
    ax.set_aspect('equal')
    ax.add_collection(lc)
    ax.autoscale()

In [107]:
# Find our Primes
@jit(nopython=True, error_model='numpy', fastmath=True, parallel=True)
def numba_sieve_of_eratosthenes(n):
    primes = [True for i in range(n+1)] # Start assuming all numbers are primes
    primes[0] = False # 0 is not a prime
    primes[1] = False # 1 is not a prime
    for i in range(2,int(np.sqrt(n)) + 1):
        if primes[i]:
            k = 2
            while i*k <= n:
                primes[i*k] = False
                k += 1
    return(primes)

PRIMES = numba_sieve_of_eratosthenes(len(cities))

In [108]:
def total_distance(city_df, path):
    prev_city = path[0]
    total_distance = 0
    step_num = 1

    for city_num in path[1:]:
        next_city = city_num
        
        x2 = np.power((city_df.X[city_num] - city_df.X[prev_city]),2)
        y2 = np.power((city_df.Y[city_num] - city_df.Y[prev_city]),2)
        city1 = [city_df.X[city_num], city_df.Y[city_num]]
        city1 = [city_df.X[city_num], city_df.Y[city_num]]
        dist = np.sqrt(x2 + y2)
        
        is_prime = PRIMES[prev_city]
        is_tenth_step = step_num % 10 == 0
        
        penalty = 1 + is_tenth_step * 0.1 * (not(is_prime))
        
        total_distance += dist * penalty
        
        prev_city = next_city
        step_num += 1
        
    return total_distance

In [109]:
#Subset to our Problem
city_subset = cities[(cities.X < 500) & (cities.Y < 500)]

In [110]:
total_distance(cities, city_subset.CityId.values)

85752.48949926025

In [111]:
def random_permutation(cities, start_city=None):
    # Todo: add in end_city
    
    perm = cities.CityId.values.copy() # list of city indices
    perm = cities
    
    if start_city:
        perm = np.delete(perm, np.where(perm == start_city), axis=0) # don't want to swap start city
    
    for i in range(len(perm)):
        r = np.random.randint(0, high=len(perm)-i) + i
        perm[r], perm[i] = perm[i], perm[r] # swaps two cities
    
    if start_city:
        perm = np.insert(perm, 0, start_city, axis=0)
        
    return perm # new path with random pairs swapped

In [112]:
def is_tabu(perm, tabu_list):
    for i in range(len(perm)):
        c1 = perm[i] # city 1
        if i == len(perm)-1:
            c2 = perm[0]
        else:
            c2 = perm[i+1]
    
        if [c1,c2] in tabu_list:
            return True
  
    return False

In [113]:
#### FROM THE WIKIPEDIA 2-OPT PAGE

def two_opt_optimization(cities, current_path):
    # TODO: Add in Kernighan–Lin algorithm
    # https://en.wikipedia.org/wiki/Kernighan%E2%80%93Lin_algorithm
    best_distance = total_distance(cities, current_path)
    
    for i in range(1, len(current_path)-2):
        for k in range(i+1, len(current_path)-1):
            new_path = two_opt_swap(current_path, i, k)
            new_distance = total_distance(cities, new_path)
            if (new_distance < best_distance):
                
                return new_path, new_distance
    return current_path, best_distance

def two_opt_swap(path, i, k):
    new_path = path[:]
    new_path[i:k] = path[k-1:i-1:-1]
    return new_path

In [114]:
#### FROM CLEVER ALGORITHMS

def stochastic_two_opt(parent):
    
    perm = parent # SHOULD THIS BE A COPY OF current_path OR AN EMPTY ARRAY LEN OF c_p?
    
    c1 = np.random.randint(len(perm))
    c2 = np.random.randint(len(perm))
    
    exclude = [c1]
    
    if c1 == 0:
        exclude.append(len(perm)-1)
    else:
        exclude.append(c1-1)
        
    if c1 == len(perm)-1:
        exclude.append(0)
    else:
        exclude.append(c1+1)
    
    while c2 in exclude:
        c2 = np.random.randint(len(perm))
        if c2 < c1:
            c1, c2 = c2, c1 # now c1 < c2
            perm = two_opt_swap(perm, c1, c2)
    
    return perm, [ [ parent[c1-1], parent[c1] ], [ parent[c2-1], parent[c2] ] ] # permuted path, edges that were swapped

In [115]:
a = range(10)

In [116]:
def generate_candidate(best_path, tabu_list, cities):

    perm = best_path
    edges = [[0,0]], [[0,0]]
    
    tabu = True
    while tabu:
        print(tabu)
        perm, edges = stochastic_two_opt(best_path)
        tabu = is_tabu(perm, tabu_list)
    
    perm_distance = total_distance(cities, perm)
    return [perm_distance, perm, edges]

In [117]:
def get_best_candidate(candidates):
    cost, best = None, None
    
    for perm_distance, perm, edges in candidates:
        if cost is None:
            cost = perm_distance
            best = [perm_distance, perm, edges]
            
        if perm_distance < cost:
            best = [perm_distance, perm, edges]
            
    return best

In [118]:
def tabu_search(cities, tabu_list_size, max_candidates, max_iter, start_city):
    best_path = random_permutation(cities, start_city)
    best_cost = total_distance(cities, best_path)
    
    tabu_list = []
    
    for i in range(max_iter):
        candidates = []
        for j in range(max_candidates):
            candidates.append(generate_candidate(best_path, tabu_list, cities))
            best_candidate = get_best_candidate(candidates)
            
            candidate_cost = best_candidate[0]
            candidate_path = best_candidate[1]
            candidate_edges = best_candidate[2]
            
            if candidate_cost < best_cost:
                best_cost = candidate_cost
                best_path = candidate_path
                
                for edge in candidate_edges:
                    tabu_list.append(edge)
                
                while len(tabu_list) > tabu_list_size:
                    tabu_list.pop(0)
        
    return best_cost, best_path
    
    # sort candidates by candidate_distance

In [119]:
# Intialize
max_iter = 1
tabu_list_size = 15
max_candidates = 10 #len(city_subset)

tabu_search(cities, tabu_list_size, max_candidates, max_iter, 3)

True


KeyboardInterrupt: 